In [2]:
import pandas as pd

<h1> Part 1(a): Employee with greater salary than their manager </h1>

In [4]:
empl_df = pd.read_csv("data/employee_test.csv")
empl_df.head()

,id,name,salary,manager_id
0,1,John,300,3.0
1,2,Mike,200,3.0
2,3,Sally,550,4.0
3,4,Jane,500,7.0
4,5,Joe,600,7.0


In [ ]:
def emplGreaterSalary(empl_df):
    """
    Retrieve list of employees with salary grater than their manager

    Arguments:
    empl_df -- employee information, dataframe with 4 columns

    Return:
    greater_empl_names -- list of employees, list
    empl_mgr_df -- employee and their respective manager's information, dataframe with 6 columns 
    """

    # Create a new table with both employee and manager information
    empl_mgr_df = empl_df.merge(empl_df[['id', 'salary']],
                        how='left',
                        left_on='manager_id',
                        right_on='id',
                        suffixes=('_empl', '_mgr'))
    
    # Apply filter where an employee's salary is larger than their immediate manager
    greater_empl_df = empl_mgr_df[empl_mgr_df['salary_empl'] > empl_mgr_df['salary_mgr']]

    # Retrieve the names of employees fulfilling the above filter
    greater_empl_names = greater_empl_df['name'].to_list()

    # Sanity assertions
    assert empl_mgr_df.shape[0] == empl_df.shape[0], "Mismatched in DataFrame sizes."

    return greater_empl_names, empl_mgr_df


In [ ]:
greater_empl_names, _ = emplGreaterSalary(empl_df)
print("People with salaries greater than their immediate manager:", 
        ", ".join(greater_empl_names))

<h1> Part 1(b): Average salary of non-managers </h1>

In [5]:
def nonMgrSalary(empl_df):
    """
    Compute the average salary of non-managers

    Arguments:
    empl_df -- employee information, dataframe with 4 columns

    Return:
    avg_non_mgr_salary -- average salary of non-managers, float 2 decimals
    non_mgr_df -- information of non-manager employee, dataframe with 4 columns 
    """

    # Get a list of employee IDs who are managers
    mgr_list = empl_df['manager_id'].dropna().unique()

    # Apply filter where employee ID is not in managers list
    non_mgr_df = empl_df[~empl_df['id'].isin(mgr_list)]

    # Take the average of the non-managers salary
    avg_non_mgr_salary = non_mgr_df['salary'].mean()

    return round(avg_non_mgr_salary, 2)

In [7]:
avg_non_mgr_salary = nonMgrSalary(empl_df)
print("Average salary of employees who do not manage anyone:", avg_non_mgr_salary)

Average salary of employees who do not manage anyone: 425.0
